In [1]:
import pandas as pd

In [4]:
open_spaces = pd.read_csv('data/locations/open_spaces.csv')
open_spaces = open_spaces.loc[~open_spaces['Type'].isin(['Cemeteries & Burying Grounds', 'Urban Wilds', 'Open Land', 'Other Open Land'])]
canopy = pd.read_csv('data/locations/open_spaces_with_canopy.csv')
streetlights = pd.read_csv('data/locations/open_spaces_with_streetlights.csv')
nearest_bluebike_stations = pd.read_csv('nearest_bluebike_stations_k_10_d_0.5.csv')
nearest_restaurants = pd.read_csv('nearest_restaurants_k_10_d_0.5.csv')
nearest_food_trucks = pd.read_csv('nearest_food_trucks_k_10_d_0.5.csv')
nearest_farmers_markets = pd.read_csv('nearest_farmers_markets_k_7_d_0.5.csv')
nearest_murals = pd.read_csv('nearest_murals_k_10_d_0.5.csv')
roundtrips = pd.read_csv('roundtrips.csv')
park_assets = pd.read_csv('data/locations/park_assets.csv')

In [5]:
selected = [
    'Boston Common',
    'Chinatown Park',
    'City Hall Plaza',
    'Dewey Square Parks',
    'Fallon Field',
    'Faneuil Square',
    'Olmsted Park',
    'Ronan Park',
    'Ross Playground',
    'Carson Beach'
]

In [6]:
open_spaces = open_spaces.loc[open_spaces['Name'].isin(selected)]
open_spaces

,SPACEID,Name,Type,Acres,Geometry,Avg. Lat,Avg. Long,Lat Range,Long Range
24,25,Boston Common,Parks,45.740950,MULTIPOLYGON (((-71.06325725499994 42.35760757...,42.355043,-71.065667,0.005130,0.007050
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,MULTIPOLYGON (((-71.05865943099997 42.36071787...,42.360502,-71.058934,0.002333,0.002581
121,122,Fallon Field,Athletic Fields,7.512146,MULTIPOLYGON (((-71.13383823799995 42.28732795...,42.287572,-71.135249,0.001399,0.003204
159,160,Ronan Park,Parks,11.241366,MULTIPOLYGON (((-71.06341298199999 42.30565509...,42.304354,-71.062631,0.002601,0.003665
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,MULTIPOLYGON (((-71.05648443799998 42.36033046...,42.360045,-71.056464,0.000571,0.001081
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,MULTIPOLYGON (((-71.05865915699997 42.35217557...,42.351329,-71.059101,0.002085,0.001535
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,MULTIPOLYGON (((-71.05163775199998 42.35475411...,42.354103,-71.053536,0.001998,0.003796
423,424,Olmsted Park,Parks,42.825230,MULTIPOLYGON (((-71.11463093299994 42.32455331...,42.325980,-71.115734,0.009777,0.007450
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,MULTIPOLYGON (((-71.03837250899994 42.32975120...,42.326421,-71.041152,0.007558,0.015440
483,484,Ross Playground,Playgrounds,13.124954,MULTIPOLYGON (((-71.11254901299998 42.26544443...,42.264119,-71.112935,0.002650,0.003789


In [7]:
park_assets['Feature List'] = park_assets['Feature(s)'].apply(str.split, sep=', ')
playgrounds = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Playground' in x), 'Park'])
athletic_fields = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Athletic Field' in x), 'Park'])

In [8]:
data = open_spaces.loc[:, 'SPACEID':'Acres']

data['Canopy Cover Proportion'] = canopy['Canopy Cover Proportion']

data['Streetlight Proportion'] = streetlights['Streetlight Cover Proportion']

data['Number of Nearby Bluebike Stations'] = nearest_bluebike_stations['num_objs']
data[['Daily Roundtrips', 'Weighted Roundtrip Ratio']] = roundtrips[['Daily Roundtrips', 'Weighted Roundtrip Ratio']]

data['Number of Nearby Restaurants'] = (nearest_restaurants['num_objs']).astype(int)
data['Near Food Trucks'] = nearest_food_trucks['num_objs'] > 0
data['Near Farmers Markets'] = nearest_farmers_markets['num_objs'] > 0

data['Has Athletic Field'] = data['Name'].isin(athletic_fields)
data['Has Playground'] = data['Name'].isin(playgrounds)

data['Near Murals'] = nearest_murals['num_objs'] > 0

data

,SPACEID,Name,Type,Acres,Canopy Cover Proportion,Streetlight Proportion,Number of Nearby Bluebike Stations,Daily Roundtrips,Weighted Roundtrip Ratio,Number of Nearby Restaurants,Near Food Trucks,Near Farmers Markets,Has Athletic Field,Has Playground,Near Murals
24,25,Boston Common,Parks,45.740950,0.357298,0.965003,10,24.004219,0.053475,10,False,False,True,True,True
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,0.129559,0.870738,10,12.525618,0.029164,10,True,False,False,True,True
121,122,Fallon Field,Athletic Fields,7.512146,0.145290,0.758661,3,0.857143,0.130357,10,False,True,True,True,True
159,160,Ronan Park,Parks,11.241366,0.221674,0.860189,2,0.840265,0.102920,10,False,False,True,True,True
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,0.013949,1.000000,8,12.150693,0.035247,10,True,False,False,False,True
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,0.512517,1.000000,6,9.386377,0.028464,10,False,False,False,True,True
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,0.422465,0.959966,7,8.569620,0.023023,10,False,False,False,False,True
423,424,Olmsted Park,Parks,42.825230,0.612260,0.297061,2,1.450874,0.039661,10,False,False,True,False,True
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,0.112254,0.311006,3,5.060880,0.142281,8,False,False,False,False,True
483,484,Ross Playground,Playgrounds,13.124954,0.177787,0.349196,1,0.105485,0.551181,3,False,False,True,True,True


In [9]:
data['Environment Score'] = data['Acres'] + data['Canopy Cover Proportion']
data['Safety Score'] = data['Streetlight Proportion']
data['Activity Score'] = data['Number of Nearby Bluebike Stations'] + data['Daily Roundtrips'] + data['Weighted Roundtrip Ratio']
data['Food Score'] = data['Number of Nearby Restaurants'] + data['Near Food Trucks'] + data['Near Farmers Markets']
data['Play Score'] = data['Has Athletic Field'].astype(int) + data['Has Playground'].astype(int)
data

,SPACEID,Name,Type,Acres,Canopy Cover Proportion,Streetlight Proportion,Number of Nearby Bluebike Stations,Daily Roundtrips,Weighted Roundtrip Ratio,Number of Nearby Restaurants,Near Food Trucks,Near Farmers Markets,Has Athletic Field,Has Playground,Near Murals,Environment Score,Safety Score,Activity Score,Food Score,Play Score
24,25,Boston Common,Parks,45.740950,0.357298,0.965003,10,24.004219,0.053475,10,False,False,True,True,True,46.098248,0.965003,34.057694,10,2
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,0.129559,0.870738,10,12.525618,0.029164,10,True,False,False,True,True,6.143529,0.870738,22.554782,11,1
121,122,Fallon Field,Athletic Fields,7.512146,0.145290,0.758661,3,0.857143,0.130357,10,False,True,True,True,True,7.657436,0.758661,3.987500,11,2
159,160,Ronan Park,Parks,11.241366,0.221674,0.860189,2,0.840265,0.102920,10,False,False,True,True,True,11.463041,0.860189,2.943185,10,2
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,0.013949,1.000000,8,12.150693,0.035247,10,True,False,False,False,True,0.830738,1.000000,20.185940,11,0
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,0.512517,1.000000,6,9.386377,0.028464,10,False,False,False,True,True,1.354490,1.000000,15.414841,10,1
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,0.422465,0.959966,7,8.569620,0.023023,10,False,False,False,False,True,3.062914,0.959966,15.592644,10,0
423,424,Olmsted Park,Parks,42.825230,0.612260,0.297061,2,1.450874,0.039661,10,False,False,True,False,True,43.437490,0.297061,3.490535,10,1
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,0.112254,0.311006,3,5.060880,0.142281,8,False,False,False,False,True,24.758651,0.311006,8.203161,8,0
483,484,Ross Playground,Playgrounds,13.124954,0.177787,0.349196,1,0.105485,0.551181,3,False,False,True,True,True,13.302741,0.349196,1.656666,3,2


In [10]:
data.to_csv('final_data_table.csv', index=False)